In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# Import

In [2]:
from db.mongo_connection import MyMongo
from util.util import save_df_to_tsv

# Mongo Connection

In [3]:
with MyMongo() as db:
    conference = db.get_df_from_table('assembly', 'watch_conference')
    rollbook = db.get_df_from_table('assembly', 'watch_rollbook')
    member = db.get_df_from_table('assembly', 'watch_member')

<--Mongo Connected.
Mongo Connection Closed.-->


# conference

In [4]:
conference = conference.drop(columns=['_id'])
print(len(conference))
conference

1290


,committee_id,committee_round,committeename,confer_date,confer_id,rollbook_fetched
0,보건복지위원회,제343회 02차,보건복지위원회,2016-06-21,9002,True
1,환경노동위원회,제343회 01차,환경노동위원회,2016-06-21,9092,True
2,법제사법위원회,제343회 01차,법제사법위원회,2016-06-20,8994,True
3,농림축산식품해양수산위원회,제343회 01차,농림축산식품해양수산위원회,2016-06-20,8995,True
4,산업통상자원중소벤처기업위원회,제343회 01차,산업통상자원중소벤처기업위원회,2016-06-20,8996,True
5,국토교통위원회,제343회 01차,국토교통위원회,2016-06-20,8997,True
6,정보위원회,제343회 01차,정보위원회,2016-06-20,8998,True
7,교육문화체육관광위원회,제343회 01차,교육문화체육관광위원회,2016-06-17,8992,True
8,여성가족위원회,제343회 01차,여성가족위원회,2016-06-17,8993,True
9,보건복지위원회,제343회 01차,보건복지위원회,2016-06-15,8991,True


# rollbook

In [5]:
rollbook = rollbook.drop(columns=['_id'])
print(len(rollbook))
rollbook

28208


,attended,confer_id,empNm,party,seq
0,출석,9002,김광수,국민의당,1013
1,출석,9002,천정배,국민의당,246
2,출석,9002,최도자,국민의당,960
3,출석,9002,권미혁,더불어민주당,946
4,출석,9002,기동민,더불어민주당,885
5,출석,9002,김상희,더불어민주당,552
6,출석,9002,남인순,더불어민주당,833
7,출석,9002,양승조,더불어민주당,402
8,출석,9002,오제세,더불어민주당,406
9,출석,9002,인재근,더불어민주당,712


# grb_attendance

In [6]:
grb_attendance = rollbook.groupby(['empNm', 'seq', 'attended']).agg({'seq': 'count'}); grb_attendance
grb_attendance = grb_attendance.unstack(); grb_attendance
grb_attendance.columns = grb_attendance.columns.droplevel(); grb_attendance
grb_attendance.columns.name = ''
grb_attendance = grb_attendance.reset_index(); grb_attendance
grb_attendance = grb_attendance.fillna(0); grb_attendance
grb_attendance.iloc[:, 2:6] = grb_attendance.iloc[:, 2:6].applymap(lambda x: int(x)); grb_attendance
grb_attendance['합계'] = grb_attendance['결석'] + grb_attendance['청가'] + grb_attendance['출석'] + grb_attendance['출장']; grb_attendance
grb_attendance['결석률'] = grb_attendance['결석'] / grb_attendance['합계'] * 100; grb_attendance
grb_attendance['출석률'] = grb_attendance['출석'] / grb_attendance['합계'] * 100; grb_attendance
grb_attendance = grb_attendance.merge(member[['seq', '정당', '당선횟수']], on='seq', how='left'); grb_attendance

,empNm,seq,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수
0,강길부,318,23,5,85,0,113,20.353982,75.221239,무소속,"4선 [ 17대, 18대, 19대, 20대 ]"
1,강병원,889,6,6,112,0,124,4.838710,90.322581,더불어민주당,1선
2,강석진,972,13,0,82,0,95,13.684211,86.315789,자유한국당,1선
3,강석호,537,16,0,69,0,85,18.823529,81.176471,자유한국당,"3선 [ 18대, 19대, 20대 ]"
4,강창일,320,17,4,73,0,94,18.085106,77.659574,더불어민주당,"4선 [ 17대, 18대, 19대, 20대 ]"
5,강효상,935,25,13,65,0,103,24.271845,63.106796,자유한국당,1선
6,강훈식,1005,5,3,74,0,82,6.097561,90.243902,더불어민주당,1선
7,경대수,755,13,0,77,0,90,14.444444,85.555556,자유한국당,"2선 [ 19대, 20대 ]"
8,고용진,887,4,5,65,0,74,5.405405,87.837838,더불어민주당,1선
9,곽대훈,911,19,0,78,0,97,19.587629,80.412371,자유한국당,1선


## Find NaN

In [7]:
att_nans = grb_attendance.loc[grb_attendance['정당'].isna()]
nans = {
    '권석창': ['자유한국당', '상실, 선거법위반', '1선'],
    '김경수': ['더불어민주당', '사퇴, 경남도지사후보', '1선'],
    '김종인': ['더불어민주당', '사퇴, 탈당', '5선'],
    '김종태': ['자유한국당', '상실, 선거법위반, 당시 새누리당', '2선'],
    '문미옥': ['더불어민주당', '사퇴, 대통령비서실 과학기술보좌관', '1선'],
    '박남춘': ['더불어민주당', '사퇴, 인천시장후보', '2선'],
    '박준영': ['민주평화당', '상실, 선거법위반', '1선'],
    '박찬우': ['자유한국당', '상실, 선거법위반', '1선'],
    '배덕광': ['자유한국당', '사퇴, 엘시티게이트', '2선'],
    '송기석': ['바른미래당', '상실, 선거법위반', '1선'],
    '안철수': ['바른미래당', '사퇴, 대통령후보', '2선'],
    '양승조': ['더불어민주당', '사퇴, 충남도지사후보', '4선'],
    '윤종오': ['민중당', '상실, 선거법위반, 당시 무소속', '1선'],
    '이철우': ['자유한국당', '사퇴, 경북도지사후보', '3선'],
    '최명길': ['바른미래당', '상실, 선거법위반', '1선'],
}
for key, value in nans.items():
    grb_attendance.loc[grb_attendance['empNm']==key, '정당'] = value[0]
    grb_attendance.loc[grb_attendance['empNm']==key, '특이사항'] = value[1]
    grb_attendance.loc[grb_attendance['empNm']==key, '당선횟수'] = value[2]

'\n권석창: 자한당(상실, 선거법위반)\n김경수: 더민주(사퇴, 경남도지사후보)\n김종인: 더민주(사퇴, 탈당)\n김종태: 자한당(상실, 선거법위반)\n문미옥: 더민주(사퇴, 대통령비서실 과학기술보좌관)\n박남춘: 더민주(사퇴, 인천시장후보)\n박준영: 민평당(상실, 선거법위반)\n박찬우: 자한당(상실, 선거법위반)\n배덕광: 자한당(사퇴, 엘시티게이트)\n송기석: 국민의당(상실, 선거법위반)\n안철수: 국민의당(사퇴, 대통령후보)\n양승조: 더민주(사퇴, 충남도지사후보)\n윤종오: 무소속(상실, 선거법위반)\n이철우: 자한당(사퇴, 경북도지사후보)\n최명길: 국민의당(상실, 선거법위반)\n'

# member

In [8]:
member = member.drop(columns=['_id']); member

,Email,empNm,hjNm,seq,당선횟수,선거구,소속위원회,연락처,정당,주요경력,학력
0,radiohaha@naver.com,하태경,河泰慶,727,"2선 [ 19대, 20대 ]",부산 해운대구갑,환경노동,의원회관 939호 | 전화 02-784-2491 | 팩스 02-788-0382,바른미래당,새누리당 북한인권 및 탈북자납북자위원회 위원장 | 민주평화통일자문회의 자문위원 | ...,브니엘고등학교 | 서울대학교 물리학과 | 고려대학교국제대학원 석사
1,dreamham7@na.go.kr,함진규,咸珍圭,795,"2선 [ 19대, 20대 ]",경기 시흥시갑,국토교통 | 예산결산특별,의원회관 547호 | 전화 02-784-4277 | 팩스 02-788-0387,자유한국당,새누리당 대변인 | 한나라당 수석부대변인 | 경기도의회 의원,고려대학교대학원 법학 박사 수료
2,7882971@hanmail.net,홍의락,洪宜洛,843,"2선 [ 19대, 20대 ]",대구 북구을,산업통상자원중소벤처기업 | 예산결산특별 | 4차 산업혁명 특별,의원회관 617호 | 전화 02-784-6277 | 팩스 02-788-0392,더불어민주당,민주당 경상북도당위원장 | 열린정책연구원 이사,고려대학교 농업경제학과
3,peace@peacehong.net,홍익표,洪翼杓,708,"2선 [ 19대, 20대 ]",서울 중구성동구갑,산업통상자원중소벤처기업,의원회관 839호 | 전화 02-784-6887 | 팩스 02-788-0393,더불어민주당,민주당 원내대변인 | 인천광역시 남북교류협력위원 | 북한대학원대학교 겸임교수 | 통...,한양대학교대학원 정치학 박사
4,pv21c@chol.com,황주홍,黃柱洪,769,"2선 [ 19대, 20대 ]",전남 고흥군보성군장흥군강진군,농림축산식품해양수산 | 예산결산특별,의원회관 919호 | 전화 02-784-8834 | 팩스 02-788-0399,민주평화당,아태평화재단 부총장 겸 연구실장 | 건국대학교 정치외교학과 교수 | 전라남도 강진군 군수,광주제일고등학교 | 연세대학교 정치외교학과 | 연세대학교대학원 정치학 석사 | 미국...
5,hansunkyo@assembly.go.kr,한선교,韓善敎,475,"4선 [ 17대, 18대, 19대, 20대 ]",경기 용인시병,교육문화체육관광,의원회관 913호 | 전화 02-784-2066 | 팩스 02-788-0385,자유한국당,"한나라당 홍보기획본부장 | 제17,18대 국회의원 | 한나라당 대변인 | MBC 아나운서",대일고등학교 | 성균관대학교 물리학과 | 성균관대학교국가전략대학원 정치학 석사
6,cma2008@naver.com,추미애,秋美愛,256,"5선 [ 15대, 16대, 18대, 19대, 20대 ]",서울 광진구을,외교통일,의원회관 501호 | 전화 02-784-1270 | 팩스 02-788-0381,더불어민주당,더불어민주당 최고위원 | 법무법인아주 대표변호사 | 새천년민주당 최고위원 | 광주고...,경북여자고등학교 | 한양대학교 법학과 | 한양대학교대학원 법학 석사 수료 | 연...
7,youngvote@hanmail.net,홍영표,洪永杓,680,"3선 [ 18대, 19대, 20대 ]",인천 부평구을,환경노동 | 국회운영 | 정보,의원회관 1004호 | 전화 02-784-3143 | 팩스 02-788-0391,더불어민주당,더불어민주당 인천시당 위원장 | 민주당 원내대변인 | 재정경제부 FTA국내대책 본부...,동국대학교 철학과 | 동국대학교대학원 행정학 석사 | 동국대학교대학원 행정학 박사 수료
8,mjhong2004@na.go.kr,홍문종,洪文鐘,266,"4선 [ 15대, 16대, 19대, 20대 ]",경기 의정부시을,외교통일,의원회관 848호 | 전화 02-784-4777 | 팩스 02-788-0389,자유한국당,새누리당 사무총장 | 경민대학교 총장 | 한나라당 경기도당위원장,대광고등학교 | 고려대학교 교육학과 | 고려대학교대학원 교육학 석사 | 하버드대학교...
9,mphonglove@hanmail.net,홍문표,洪文杓,477,"3선 [ 17대, 19대, 20대 ]",충남 홍성군예산군,농림축산식품해양수산,의원회관 336호 | 전화 02-784-9587 | 팩스 02-788-0390,자유한국당,새누리당 제1사무부총장 | 한국농업경영인중앙연합회 명예회원 | 한국농어촌공사 사장,한영고등학교 | 건국대학교 | 한양대학교 대학원 행정학 석사


# view_attendance

In [9]:
view_attendance = grb_attendance.drop(columns=['seq']); view_attendance
view_attendance = view_attendance.rename(columns={'empNm':'의원'}); view_attendance

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항
0,강길부,23,5,85,0,113,20.353982,75.221239,무소속,"4선 [ 17대, 18대, 19대, 20대 ]",NaN
1,강병원,6,6,112,0,124,4.838710,90.322581,더불어민주당,1선,NaN
2,강석진,13,0,82,0,95,13.684211,86.315789,자유한국당,1선,NaN
3,강석호,16,0,69,0,85,18.823529,81.176471,자유한국당,"3선 [ 18대, 19대, 20대 ]",NaN
4,강창일,17,4,73,0,94,18.085106,77.659574,더불어민주당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN
5,강효상,25,13,65,0,103,24.271845,63.106796,자유한국당,1선,NaN
6,강훈식,5,3,74,0,82,6.097561,90.243902,더불어민주당,1선,NaN
7,경대수,13,0,77,0,90,14.444444,85.555556,자유한국당,"2선 [ 19대, 20대 ]",NaN
8,고용진,4,5,65,0,74,5.405405,87.837838,더불어민주당,1선,NaN
9,곽대훈,19,0,78,0,97,19.587629,80.412371,자유한국당,1선,NaN


# sort by 결석률

In [10]:
view_sort_by_abs_rate = view_attendance.sort_values('결석률', ascending=False).reset_index().drop(columns=['index'])
view_sort_by_abs_rate
# 1위 배덕광: 자한당, 엘시티 게이트, 사직
# 18위 김종태: 자한당(새누리), 선거법위반 의원직 상실
# 29위 이철우: 자한당, 경북도지사

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항
0,배덕광,36,1,29,0,66,54.545455,43.939394,자유한국당,2선,"사퇴, 엘시티게이트"
1,서청원,37,0,35,0,72,51.388889,48.611111,자유한국당,"8선 [ 11대, 13대, 14대, 15대, 16대, 18대, 19대, 20대 ]",NaN
2,한선교,39,0,42,0,81,48.148148,51.851852,자유한국당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN
3,홍문표,38,0,45,0,83,45.783133,54.216867,자유한국당,"3선 [ 17대, 19대, 20대 ]",NaN
4,우상호,43,1,51,0,95,45.263158,53.684211,더불어민주당,"3선 [ 17대, 19대, 20대 ]",NaN
5,이종걸,35,1,47,0,83,42.168675,56.626506,더불어민주당,"5선 [ 16대, 17대, 18대, 19대, 20대 ]",NaN
6,조원진,32,1,44,0,77,41.558442,57.142857,대한애국당,"3선 [ 18대, 19대, 20대 ]",NaN
7,김무성,21,0,30,0,51,41.176471,58.823529,자유한국당,"6선 [ 15대, 16대, 17대, 18대, 19대, 20대 ]",NaN
8,추미애,22,25,16,0,63,34.920635,25.396825,더불어민주당,"5선 [ 15대, 16대, 18대, 19대, 20대 ]",NaN
9,이해찬,23,0,43,0,66,34.848485,65.151515,더불어민주당,"7선 [ 13대, 14대, 15대, 16대, 17대, 19대, 20대 ]",NaN


## 결석률TOP30 정당분포

In [11]:
view_sort_by_abs_rate.head(30).groupby('정당').agg({'정당':'count'})

,정당
정당,
대한애국당,1
더불어민주당,6
무소속,1
바른미래당,2
자유한국당,20


# sort by 출석률

In [12]:
view_sort_by_att_rate = view_attendance.sort_values('출석률', ascending=False).reset_index().drop(columns=['index'])
view_sort_by_att_rate
# 문미옥: 더민주, 대통령비서실 과학기술보좌관
# 양승조: 더민주, 충남도지사후보
# 박남춘: 더민주, 인천시장후보

,의원,결석,청가,출석,출장,합계,결석률,출석률,정당,당선횟수,특이사항
0,유동수,0,0,94,0,94,0.000000,100.000000,더불어민주당,1선,NaN
1,이수혁,0,0,22,0,22,0.000000,100.000000,더불어민주당,1선,NaN
2,전재수,0,0,104,0,104,0.000000,100.000000,더불어민주당,1선,NaN
3,김민기,0,0,84,0,84,0.000000,100.000000,더불어민주당,"2선 [ 19대, 20대 ]",NaN
4,조정식,0,0,60,0,60,0.000000,100.000000,더불어민주당,"4선 [ 17대, 18대, 19대, 20대 ]",NaN
5,문미옥,0,0,50,0,50,0.000000,100.000000,더불어민주당,1선,"사퇴, 대통령비서실 과학기술보좌관"
6,양승조,0,0,65,0,65,0.000000,100.000000,더불어민주당,4선,"사퇴, 충남도지사후보"
7,박남춘,0,0,70,0,70,0.000000,100.000000,더불어민주당,2선,"사퇴, 인천시장후보"
8,유성엽,0,0,81,0,81,0.000000,100.000000,민주평화당,"3선 [ 18대, 19대, 20대 ]",NaN
9,이원욱,0,0,62,0,62,0.000000,100.000000,더불어민주당,"2선 [ 19대, 20대 ]",NaN


## 출석률TOP30 정당분포

In [13]:
view_sort_by_att_rate.head(30).groupby('정당').agg({'정당':'count'})

,정당
정당,
더불어민주당,24
민주평화당,2
바른미래당,2
자유한국당,2


# 정당별 출석률

In [23]:
# 결석	청가	출석	출장	합계
view_party_att = view_attendance[['정당','청가','출장','결석','출석','합계']].groupby('정당').agg('sum'); view_party_att
view_party_att['결석률'] = view_party_att['결석'] / view_party_att['합계'] * 100; view_party_att
view_party_att['출석률'] = view_party_att['출석'] / view_party_att['합계'] * 100; view_party_att
view_party_att = view_party_att.reset_index(); view_party_att

,정당,청가,출장,결석,출석,합계,결석률,출석률
0,대한애국당,1,0,32,44,77,41.558442,57.142857
1,더불어민주당,564,1,637,10200,11402,5.586739,89.457990
2,무소속,23,0,60,294,377,15.915119,77.984085
3,민주평화당,75,0,81,1288,1444,5.609418,89.196676
4,민중당,8,0,20,118,146,13.698630,80.821918
5,바른미래당,158,0,223,2660,3041,7.333114,87.471227
6,자유한국당,387,2,1678,9024,11091,15.129384,81.363268
7,정의당,33,0,55,542,630,8.730159,86.031746


## 정당 결석률 순위

In [28]:
view_party_sort_abs = view_party_att.sort_values('결석률', ascending=False); view_party_sort_abs

,정당,청가,출장,결석,출석,합계,결석률,출석률
0,대한애국당,1,0,32,44,77,41.558442,57.142857
2,무소속,23,0,60,294,377,15.915119,77.984085
6,자유한국당,387,2,1678,9024,11091,15.129384,81.363268
4,민중당,8,0,20,118,146,13.698630,80.821918
7,정의당,33,0,55,542,630,8.730159,86.031746
5,바른미래당,158,0,223,2660,3041,7.333114,87.471227
3,민주평화당,75,0,81,1288,1444,5.609418,89.196676
1,더불어민주당,564,1,637,10200,11402,5.586739,89.457990


## 정당 출석률 순위

In [29]:
view_party_sort_att = view_party_att.sort_values('출석률', ascending=False); view_party_sort_att

,정당,청가,출장,결석,출석,합계,결석률,출석률
1,더불어민주당,564,1,637,10200,11402,5.586739,89.457990
3,민주평화당,75,0,81,1288,1444,5.609418,89.196676
5,바른미래당,158,0,223,2660,3041,7.333114,87.471227
7,정의당,33,0,55,542,630,8.730159,86.031746
6,자유한국당,387,2,1678,9024,11091,15.129384,81.363268
4,민중당,8,0,20,118,146,13.698630,80.821918
2,무소속,23,0,60,294,377,15.915119,77.984085
0,대한애국당,1,0,32,44,77,41.558442,57.142857


# Save

## view_attendance

In [33]:
# save_df_to_tsv(view_attendance, '../data/20대국회상임위출석부.tsv')

## view_sort_by_abs_rate

In [34]:
# save_df_to_tsv(view_sort_by_abs_rate.head(30), '../data/의원결석률순위30.tsv')

## view_sort_by_att_rate

In [35]:
# save_df_to_tsv(view_sort_by_att_rate.head(30), '../data/의원출석률순위30.tsv')

## view_party_sort_abs

In [30]:
# save_df_to_tsv(view_party_sort_abs, '../data/정당결석률순위.tsv')

## view_party_sort_att

In [31]:
# save_df_to_tsv(view_party_sort_att, '../data/정당출석률순위.tsv')